# DDPG - Deep Deterministic Policy Gradient

Deep Deterministic Policy Gradient (DDPG) é um algoritmo off-policy que aprende simultaneamente uma função Q-valor e uma política. Com a equação de Bellman, aprende a função Q e, com esta, aprende a política.

A ideia de um algoritmo com gradiente da política surgiu primeiro com [Silver (sempre ele)](http://proceedings.mlr.press/v32/silver14.pdf), mas o algoritmo DDPG em si foi estabelecido na publicação [por Lilicrap](https://arxiv.org/abs/1509.02971).

## O algoritmo de DDPG

Este algoritmo é um Actor-Critic, mas, diferentemente dos que vimos antes, é off policy. Diferente do que estamos acostumados, este algoritmo só pode ser usado em espaços de ação contínuos. Podemos pensar então no DDPG como uma "DQN para espaço de ação contínuo".

$$a^*(s) = arg max_a Q^*(s,a)$$

A equação acima, da ação ótima para um dado estado, é familiar, mas se torna problemática para ações não discretas. É fácil contabilizar o máximo de uma série finita de ações, mas ações contínuas tornam essa operação problemática.

Como sabemos que o espaço de ações é contínuo, podemos definir a política (ou função ator) $\mu_\theta(s) $, que vai nos ajudar a computar o max. Lembrando que, nesse algoritmo, a política é determinística.

Dessa forma, temos então $max_a Q(s,a) \approx Q(s, \mu(s))$, facilitando a computação do max.

### Q-Learning

$$Q^*(s,a) = E_{s \sim P} \bigg[r(s,a) + \gamma max_{a'} Q^*(s',a') \bigg] $$

A parte de Q learning, cuja equação para encontrar o valor ótimo de Q está acima, consiste em minimizar o erro quadrático médio de Bellman (MSBE), próximo do que já se era feito com a DQN.

O [post da OpenAI](https://spinningup.openai.com/en/latest/algorithms/ddpg.html) oferece duas ferramentas para facilitar o treinamento da função Q, o uso de Replay Buffers e o uso de Target Networks. Uma vantagem de DDPG ser off-policy é permitir o uso de Replay Buffers.


### Policy Gradient

O objetivo é achar a política determinística $\mu_{\theta}(s) $ que fornece a ação que maximiza $Q(s,a)$. Diferenciando a função Q em a, é possível então realizar gradiente ascendente para maximizar a política:

$$\nabla_\theta J = \mathbb{E}_{s_t} \bigg[ \nabla_a Q(s,a|\theta^Q) |_{s = s_t, a = \mu(s_t)} \nabla_\theta \mu_\theta(s|\theta^\mu)|_{s = s_t} \bigg] $$


## Exploração

Um problema para casos de ação contínua é a exploração. Por outro lado, por se tratar de um algoritmo off-policy, podemos tratar o problema da exploração independente do algoritmo de aprendizado. Dessa forma, podemos criar uma política de exploração $\mu'$ adicionando ruído na nossa política/função ator:

$$\mu'(s) = \mu_\theta(s) + \mathcal{N} $$

No paper original, o ruído foi gerado por um processo [Ornstein-Uhlenbeck](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process). Porém, a openAI recomenda o uso de um ruído gaussiano não correlacionado e de média zero, por ser mais simples e funcionar tão bem quanto.


## Pseudocódigo

![DDPG](https://i.postimg.cc/bwfJDNFb/DDPG-1.png)


## Referências
https://spinningup.openai.com/en/latest/algorithms/ddpg.html

http://proceedings.mlr.press/v32/silver14.pdf

# Código


## Ator

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Actor(nn.Module):
    """Rede do Ator."""
    def __init__(self, observation_shape, action_shape, action_high, action_low):
        """Inicializa a rede.
        
        Parâmetros
        ----------
        observation_shape: int
        Formato do estado do ambiente.
        
        action_shape: int
        Número de ações do ambiente.
        """
        super(Actor, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.action_high = torch.from_numpy(action_high).to(self.device)
        self.action_low = torch.from_numpy(action_low).to(self.device)

        self.linear1 = nn.Linear(observation_shape, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, action_shape)

    def forward(self, state):
        """
        Calcula a probabilidade de ação para o estado atual.
        
        Parâmetros
        ----------
        state: np.array
        Estado atual.
        
        Retorna
        -------
        action: torch.Tensor
        Ações.
        """
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = torch.tanh(self.linear3(x))
        
        action = x * (self.action_high - self.action_low) / 2.0 +\
            (self.action_high + self.action_low) / 2.0

        return action

## Crítico

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Critic(nn.Module):
    """Rede do Crítico."""
    def __init__(self, observation_shape, action_shape):
        """Inicializa a rede.
        
        Parâmetros
        ----------
        observation_shape: int
        Formato do estado do ambiente.

        action_shape: int
        Número de ações do ambiente.
        """
        super(Critic, self).__init__()

        self.linear1 = nn.Linear(observation_shape, 512)
        self.linear2 = nn.Linear(512 + action_shape, 512)
        self.linear3 = nn.Linear(512, 300)
        self.linear4 = nn.Linear(300, 1)

    def forward(self, state, action):
        """
        Calcula o valor do estado atual.
        
        Parâmetros
        ----------
        state: np.array
        Estado atual.

        state: np.array
        Ação escolhida.
        
        Retorna
        -------
        q: float
        Valor da ação escolhida.
        """
        x = F.relu(self.linear1(state))
        xa_cat = torch.cat([x, action], 1)
        xa = F.relu(self.linear2(xa_cat))
        xa = F.relu(self.linear3(xa))
        q = self.linear4(xa)

        return q

## Experience Replay

In [3]:
import numpy as np

class ExperienceReplay:
    """Experience Replay Buffer para A2C."""
    def __init__(self, max_length, observation_space):
        """Cria um Replay Buffer.

        Parâmetros
        ----------
        max_length: int
            Tamanho máximo do Replay Buffer.
        observation_space: int
            Tamanho do espaço de observação.
        """
        self.index, self.length, self.max_length = 0, 0, max_length

        self.states = np.zeros((max_length, *observation_space), dtype=np.float32)
        self.actions = np.zeros((max_length), dtype=np.int32)
        self.rewards = np.zeros((max_length), dtype=np.float32)
        self.next_states = np.zeros((max_length, *observation_space), dtype=np.float32)
        self.dones = np.zeros((max_length), dtype=np.float32)

    def update(self, state, action, reward, next_state, done):
        """Adiciona uma experiência ao Replay Buffer.

        Parâmetros
        ----------
        state: np.array
            Estado da transição.
        action: int
            Ação tomada.
        reward: float
            Recompensa recebida.
        state: np.array
            Estado seguinte.
        done: int
            Flag indicando se o episódio acabou.
        """
        self.states[self.index] = state
        self.actions[self.index] = action
        self.rewards[self.index] = reward
        self.next_states[self.index] = next_state
        self.dones[self.index] = done

        self.index = (self.index + 1) % self.max_length
        if self.length < self.max_length:
            self.length += 1

    def sample(self, batch_size):
        """Retorna um batch de experiências.
        
        Parâmetros
        ----------
        batch_size: int
            Tamanho do batch de experiências.

        Retorna
        -------
        states: np.array
            Batch de estados.
        actions: np.array
            Batch de ações.
        rewards: np.array
            Batch de recompensas.
        next_states: np.array
            Batch de estados seguintes.
        dones: np.array
            Batch de flags indicando se o episódio acabou.
        """
        # Escolhe índices aleatoriamente do Replay Buffer
        idxs = np.random.randint(0, self.length, size=batch_size)

        return (self.states[idxs], self.actions[idxs], self.rewards[idxs], self.next_states[idxs], self.dones[idxs])

## Agente

In [4]:
import numpy as np
import torch
from torch import optim

class DDPG:
    def __init__(self, observation_space, action_space, pi_lr=0.001, q_lr=0.001, gamma=0.99, tau=0.005, action_noise=0.1):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.gamma = gamma
        self.tau = tau

        self.memory = ExperienceReplay(10000, observation_space.shape)

        self.action_noise = action_noise
        self.action_shape = action_space.shape
        self.action_high = action_space.high
        self.action_low = action_space.low

        self.actor = Actor(observation_space.shape[0], action_space.shape[0], self.action_high, self.action_low).to(self.device)
        self.target_actor = Actor(observation_space.shape[0], action_space.shape[0], self.action_high, self.action_low).to(self.device)

        self.critic = Critic(observation_space.shape[0], action_space.shape[0]).to(self.device)
        self.target_critic = Critic(observation_space.shape[0], action_space.shape[0]).to(self.device)

        for target_param, param in zip(self.target_critic.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)

        for target_param, param in zip(self.target_actor.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data)

        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=pi_lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=q_lr)

    def act(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            action = self.actor.forward(state)
        
        action = action.squeeze(0).cpu().numpy()
        action += self.action_noise * np.random.randn(*self.action_shape)

        return np.clip(action, self.action_low, self.action_high)

    def remember(self, state, action, reward, next_state, done):
        self.memory.update(state, action, reward, next_state, done)

    def train(self, batch_size=32, epochs=1):
        if batch_size > self.memory.length:
            return
        
        for epoch in range(epochs):
            (states, actions, rewards, next_states, dones) = self.memory.sample(batch_size)

            states = torch.FloatTensor(states).to(self.device)
            actions = torch.FloatTensor(actions).unsqueeze(-1).to(self.device)
            rewards = torch.FloatTensor(rewards).unsqueeze(-1).to(self.device)
            next_states = torch.FloatTensor(next_states).to(self.device)
            dones = torch.FloatTensor(dones).unsqueeze(-1).to(self.device)

            self._train_critic(states, actions, rewards, next_states, dones)
            self._train_actor(states, actions, rewards, next_states, dones)
            self.update_target()

    def _train_critic(self, states, actions, rewards, next_states, dones):
        q = self.critic.forward(states, actions)

        with torch.no_grad():
            a2 = self.target_actor.forward(next_states)
            q2 = self.target_critic.forward(next_states, a2)
        
        target = rewards + self.gamma * q2
        critic_loss = F.mse_loss(q, target)

        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()

    def _train_actor(self, states, actions, rewards, next_states, dones):
        policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean()
        
        self.actor_optimizer.zero_grad()
        policy_loss.backward()
        self.actor_optimizer.step()

    def update_target(self):
        with torch.no_grad():
            for target_param, param in zip(self.target_actor.parameters(), self.actor.parameters()):
                target_param.data.mul_(1 - self.tau)
                torch.add(target_param.data, param.data, alpha=self.tau, out=target_param.data)

            for target_param, param in zip(self.target_critic.parameters(), self.critic.parameters()):
                target_param.data.mul_(1 - self.tau)
                torch.add(target_param.data, param.data, alpha=self.tau, out=target_param.data)

## Treinando

In [5]:
import math
from collections import deque

def train(agent, env, total_timesteps):
    total_reward = 0
    episode_returns = deque(maxlen=20)
    avg_returns = []

    state = env.reset()
    timestep = 0
    episode = 0

    while timestep < total_timesteps:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        loss = agent.train()
        timestep += 1

        total_reward += reward

        if done:
            episode_returns.append(total_reward)
            episode += 1
            next_state = env.reset()

        if episode_returns:
            avg_returns.append(np.mean(episode_returns))

        total_reward *= 1 - done
        state = next_state

        ratio = math.ceil(100 * timestep / total_timesteps)

        avg_return = avg_returns[-1] if avg_returns else np.nan
        
        print(f"\r[{ratio:3d}%] timestep = {timestep}/{total_timesteps}, episode = {episode:3d}, avg_return = {avg_return:10.4f}", end="")

    return avg_returns

### Pendulum-v0

![Pendulum-v0](https://mspries.github.io/img/jimmy-pendulum/pendulum.gif)

In [6]:
import gym

env = gym.make("Pendulum-v0")

In [7]:
agent = DDPG(env.observation_space, env.action_space)

In [8]:
returns = train(agent, env, 15000)

[100%] timestep = 15000/15000, episode =  75, avg_return =  -129.0745